In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# pycaret

In [ ]:
!pip install pycaret

In [ ]:
from pycaret.regression import *

# read data

In [ ]:
train = pd.read_csv("../input/mercedes-benz-greener-manufacturing/train.csv.zip")
test = pd.read_csv("../input/mercedes-benz-greener-manufacturing/test.csv.zip")

In [ ]:
train.head()

In [ ]:
test.head()

# check for null to decide whether proprocess

In [ ]:
train[train.isnull().any(axis=1)]

In [ ]:
test[test.isnull().any(axis=1)]

In [ ]:
train = train.sample(frac=0.3)

# init setup

In [ ]:
reg = setup(data = train, 
            target = 'y', 
            numeric_imputation = 'mean',
            categorical_features = ['X0','X1','X2','X3','X4','X5','X6','X8'], 
            ignore_features = ['ID'],
            silent = True)

In [ ]:
models = compare_models()

In [ ]:
models

In [ ]:
gbr = create_model('gbr')

In [ ]:
tune_gbr = tune_model(gbr)

In [ ]:
plot_model(tune_gbr)

In [ ]:
plot_model(tune_gbr, plot="error")

In [ ]:
plot_model(tune_gbr, plot="learning")

In [ ]:
plot_model(tune_gbr, plot="feature")

# 4 models to blend

In [ ]:
br = create_model("br")

In [ ]:
tune_br = tune_model(br)

In [ ]:
rf = create_model('rf')

In [ ]:
tune_rf = tune_model(rf)

In [ ]:
catboost = create_model('catboost')

In [ ]:
tune_catboost = tune_model(catboost)

In [ ]:
blended = blend_models(estimator_list = [tune_gbr, tune_br, tune_rf, tune_catboost], 
                       fold = 5)

In [ ]:
predictions = predict_model(blended, data = test)
predictions.head()

# soumission

In [ ]:
sample_submission = pd.read_csv("../input/mercedes-benz-greener-manufacturing/sample_submission.csv.zip")

In [ ]:
sample_submission['Label'] = predictions['Label']
sample_submission.to_csv('submission.csv', index=False)